# Initialization

In [1]:
USE_MP = True
import silq
silq.initialize("EWJN")

qc.show_subprocess_widget()

Initializing imports


<IPython.core.display.Javascript object>

Initializing functions
Initializing instruments
Initializing layout
Initializing data
Initializing analysis
Initializing parameters
Initializing plotting
Initializing slack
Initialization complete


In [55]:
pulse = adiabatic_ESR_parameter.pulse_sequence['adiabatic']
pulse.t_start = None
pulse

FrequencyRampPulse(adiabatic (ESR), f_center=27951.00 MHz, f_dev=0.50, power=0, t_start=3, t_stop=3.2)

In [66]:
pulse_config


{'adiabatic_ESR': {'duration': 0.2,
  'frequency_deviation': 500000.0,
  'power': 0,
  't_start': 3},
 'adiabatic_NMR': {'duration': 0.5,
  'frequency_deviation': 100000.0,
  'power': 0,
  't_start': 3},
 'empty': {'amplitude': -1.5, 'duration': 5},
 'final': {'amplitude': 0, 'duration': 2},
 'plunge': {'amplitude': 2, 'duration': 5},
 'read': {'amplitude': 0, 'duration': 20},
 'read_long': {'duration': 120},
 'steered_initialization': {'enabled': False,
  't_buffer': 30,
  't_max_wait': 400,
  't_no_blip': 120},
 'steered_initialization_long': {'t_max_wait': 1000, 't_no_blip': 200}}

In [4]:
dark_counts_plunge_parameter = acquisition_parameters.T1Parameter(mode='ESR')

dark_counts_plunge_parameter.pulse_sequence['adiabatic'].enabled = False
dark_counts_plunge_parameter.pulse_sequence['steered_initialization'].enabled = True
dark_counts_plunge_parameter.pulse_sequence['steered_initialization'].mode = 'long'
dark_counts_plunge_parameter.pulse_sequence['plunge'].duration = 20
dark_counts_plunge_parameter.pulse_sequence

PulseSequence with 5 pulses, duration: 42
	SteeredInitialization(steered_initialization (long), t_no_blip=200 ms, t_max_wait=1000, t_buffer=30, V_th=0.3333333333333333)
	DCPulse(plunge, A=2, t_start=0, t_stop=20)
	DCPulse(read, A=0, t_start=20, t_stop=40)
	DCPulse(final, A=0, t_start=40, t_stop=42)
	
	Disabled pulses:
	FrequencyRampPulse(adiabatic (ESR), f_center=27951.00 MHz, f_dev=0.50, power=0, t_start=3, t_stop=3.2)

## Register code run commands

In [2]:
%%label adiabatic
samples = 100
points = 7
steps = 10

y_sweep, x_sweep = create_set_vals(2, step_percentage=steps, points=points)
adiabatic_ESR_parameter.samples = samples
data = qc.Loop(
    y_sweep).loop(
        x_sweep).each(
            adiabatic_ESR_parameter).run(
    name='adiabatic_calibration', progress_interval=True)
CalibrationPlot(data)

In [3]:
%%label dc
points = 30
steps = 25

y_sweep, x_sweep = create_set_vals(2, step_percentages=steps, points=points)
DC_parameter.setup()
data = qc.Loop(
    y_sweep).loop(
    x_sweep).each(
    DC_parameter).then(
    qc.Task(layout.stop)).run(
    name='DC_{}_vs_{}_scan'.format(y_sweep.name, x_sweep.name),
    progress_interval=True)

plot = qc.MatPlot(data.DC_voltage)

## Threshold voltage

In [ ]:
EPR_parameter.single_settings(samples=50)
EPR_parameter.setup()
voltage_difference = EPR_parameter()[-1]

assert voltage_difference, "Couldn't find accurate threshold"
readout_threshold_voltage = voltage_difference / 2

print('Threshold voltage found at {:.2f} V'.format(readout_threshold_voltage))
properties_config['readout_threshold_voltage'] = readout_threshold_voltage

## ESR frequency

In [ ]:
select_ESR_parameter.single_settings(samples=30, silent=False)
select_ESR_parameter();

# DC Measurements

## DC - Turnon
Here several gates are combined to the parameter turnon_param.

In [ ]:
turnon_vals = turnon_parameter[list(np.linspace(1, 1.6, 100))]

In [ ]:
DC_parameter.setup()
data = qc.Loop(turnon_vals
# data = qc.Loop(dummy_parameter[0:1000:1], delay=1
               ).each(DC_parameter
                      ).then(qc.Task(layout.stop)
                             ).run(name='turnon',
                                   progress_interval=True)
plot = qc.MatPlot(data.DC_voltage)

## DC - Pinchoff
This code pinches off one gate, while keeping the other gates fixed

In [ ]:
turnon_voltage = 1.6
pinchoff_vals = list(np.linspace(turnon_voltage, 0, 100))
gate = TG

turnon_parameter(turnon_voltage)
DC_parameter.setup()
data = qc.Loop(gate[pinchoff_vals]
               ).each(DC_parameter
                      ).then(qc.Task(layout.stop)
                             ).run(name='Pinchoff_{}'.format(gate.name),
                                   progress_interval=True)
plot = qc.MatPlot(data.DC_voltage)

In [ ]:
turnon_voltage = 1.3
pinchoff_vals = list(np.linspace(turnon_voltage, 0.7, 40))
gatex=LB
gatey=RB

turnon_parameter(turnon_voltage)
DC_parameter.setup()
data = qc.Loop(gatey[pinchoff_vals]
               ).loop(gatex[pinchoff_vals]
                      ).each(DC_parameter
                      ).then(qc.Task(layout.stop)
                             ).run(name='2D_Pinchoff_{}'.format(gate.name),
                                   progress_interval=True)
plot = qc.MatPlot(data.DC_voltage)


## DC - 1D scan
This script performs a 1D DC scan

In [ ]:
gate = DF_DS[list(np.linspace(0.28,0.6, 50))]

DC_parameter.setup()
data = qc.Loop(gate
              ).each(DC_parameter
                    ).then(qc.Task(layout.stop)
                          ).run(name='DC_{}_scan'.format(gate.name),
                                progress_interval=True)

plot = qc.MatPlot(data.DC_voltage)

In [33]:
DS()

0.4725

## DC - 2D scan

This measurement performs a 2D DC scan

In [47]:
#y_sweep, x_sweep = create_set_vals(2, step_percentage=5, points=250)
TGAC(0.30504)
DS(0.51036)

In [53]:
y_sweep = TGAC[list(np.arange(0.05, 0.55, 0.0025))]
x_sweep = DF[list(np.arange(0.2,0.61,0.0025))]

DC_parameter.setup()
data = Loop(
    y_sweep).loop(
        x_sweep).each(
            DC_parameter).then(
                qc.Task(layout.stop)).run(
    name='DC_{}_vs_{}_scan'.format(y_sweep.name, x_sweep.name),
    progress_interval=True)

DataSet:
data = qc.load_data('2017-01-19/#013_DC_TGAC_vs_DF_scan_16-13-51')
   mode = DataMode.PULL_FROM_SERVER
   <Type>   | <array_id> | <array.name> | <array.shape>
   Setpoint | TGAC_set   | TGAC         | (200,)
   Setpoint | DF_set     | DF           | (200, 164)
   Measured | DC_voltage | DC_voltage   | (200, 164)
started at 2017-01-19 16:13:54


In [54]:
DCPlot(data)

## DC - 2D scan with arbstudio

In [15]:
#ATTENTION: this is still additional voltage on the sim voltages!!!
#arbstudio are divided by 25(?)/20(?)
df_voltages = list(np.linspace(-6, 6, 200))
tgac_voltages = list(np.linspace(-6, 6, 200))
df_duration = 20

DC_pulse_sweep_parameter = acquisition_parameters.DCPulseSweepParameter(sweep_name='DF')
DC_pulse_sweep_parameter.pulse_settings = {'duration': df_duration, 
                                           'connection_requirements': {'output_arg': 'arbstudio.ch2'}}
#'pulse_modifiers': {'amplitude_scale': 1}

DC_pulse_sweep_parameter.additional_pulses = [DCPulse(name='TGAC', amplitude=0, t_start=0, duration=len(df_voltages)*df_duration+2, 
                                                      connection_requirements={'output_arg': 'arbstudio.ch1'})]

%time DC_pulse_sweep_parameter.sweep_voltages = df_voltages
DC_pulse_sweep_parameter.samples = 1

tgac_step_parameter = general_parameters.AttributeParameter(DC_pulse_sweep_parameter.pulse_sequence['TGAC'], 'amplitude')

#DC_pulse_sweep_parameter.pulse_sequence

Wall time: 17 ms


### 2D scan

In [16]:
data = qc.Loop(
    tgac_step_parameter[tgac_voltages]).each(
        DC_pulse_sweep_parameter).run(
        #qc.Task(partial(print_attr, DC_pulse_sweep_parameter, 'pulse_sequence'))).run(
    name='DC_2D_arb_sweep')

DataSet:
data = qc.load_data('2017-01-18/#005_DC_2D_arb_sweep_18-48-33')
   mode = DataMode.PULL_FROM_SERVER
   <Type>   | <array_id>    | <array.name> | <array.shape>
   Setpoint | amplitude_set | amplitude    | (200,)
   Measured | DC_voltage    | DC_voltage   | (200, 200)
started at 2017-01-18 18:48:38


In [17]:
qc.MatPlot(data.DC_voltage)

### 4D scan (sweep simrack TGAC and DF)

In [7]:
data = qc.Loop(
    TGAC[0.2:0.71:0.1]).loop(
    DF[0.2:0.71:0.1]).loop(
    tgac_step_parameter[tgac_voltages]).each(
        DC_pulse_sweep_parameter).run(
        #qc.Task(partial(print_attr, DC_pulse_sweep_parameter, 'pulse_sequence'))).run(
    name='DC_2D_arb_sweep')

DataSet:
data = qc.load_data('2017-01-18/#002_DC_2D_arb_sweep_13-39-57')
   mode = DataMode.PULL_FROM_SERVER
   <Type>   | <array_id>    | <array.name> | <array.shape>
   Setpoint | TGAC_set      | TGAC         | (6,)
   Setpoint | DF_set        | DF           | (6, 6)
   Setpoint | amplitude_set | amplitude    | (6, 6, 100)
   Measured | DC_voltage    | DC_voltage   | (6, 6, 100, 100)
started at 2017-01-18 13:40:04


In [8]:
data.DC_voltage

DataArray[6,6,100,100]: DC_voltage
array([[[[ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan],
         ..., 
         [ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

        [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan],
         ..., 
         [ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

        [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan],
         ..., 
         [ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,  nan,  nan,  nan],
         [ nan,  nan,  nan, ...,

In [4]:
dummy_parameter = ManualParameter(name='dummy')
dummy_parameters = ManualParameter(name='dummy2')

qc.Loop(dummy_parameter[0:10:1]).loop(
    dummy_parameters[0:10:1]).each(dummy_parameter).run(name='test', background=False)

is parameter array
here
also here
DataSet:
data = qc.load_data('2017-01-17/#004_test_11-32-41')
   mode = DataMode.LOCAL
   <Type>   | <array_id> | <array.name> | <array.shape>
   Setpoint | dummy_set  | dummy        | (10,)
   Setpoint | dummy2_set | dummy2       | (10, 10)
   Measured | dummy      | dummy        | (10, 10)
started at 2017-01-17 11:32:43


DataSet:
data = qc.load_data('2017-01-17/#004_test_11-32-41')
   mode = DataMode.LOCAL
   <Type>   | <array_id> | <array.name> | <array.shape>
   Setpoint | dummy_set  | dummy        | (10,)
   Setpoint | dummy2_set | dummy2       | (10, 10)
   Measured | dummy      | dummy        | (10, 10)

In [ ]:
%debug

> c:\users\lab.blug12meas\documents\silq\qcodes\qcodes\loops.py(585)_parameter_arrays()
    583             for j, (vij, nij, lij) in enumerate(zip(sp_vi, sp_ni, sp_li)):
    584                 sp_def = (shape[: 1 + j], j, setpoints, vij, nij, lij)
--> 585                 if sp_def not in all_setpoints:
    586                     all_setpoints[sp_def] = self._make_setpoint_array(*sp_def)
    587                     out.append(all_setpoints[sp_def])

ipdb> vij
array([ 1.  ,  1.15,  1.3 ,  1.45,  1.6 ])
ipdb> nij
'TGAC'
ipdb> lij
'TGAC'
ipdb> sp
*** NameError: name 'sp' is not defined
ipdb> shape
(5,)
ipdb> sp_def
((5,), 0, (), array([ 1.  ,  1.15,  1.3 ,  1.45,  1.6 ]), 'TGAC', 'TGAC')


In [8]:
data.sync()
data.DC_voltage

DataArray[5,5]: DC_voltage
array([[ 0.558,  1.033,  1.25 ,  1.364,  1.133],
       [ 0.551,  1.03 ,  1.251,  1.364,  1.157],
       [ 0.576,  1.036,  1.245,  1.368,  1.159],
       [ 0.548,  1.046,  1.256,  1.373,  1.16 ],
       [ 0.541,  1.02 ,  1.239,  1.375,  1.248]])

In [17]:
data['ac']

DataArray[10]: DC_voltage
array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan])

In [19]:
DC_pulse_sweep_parameter.trace_segments

In [20]:
plot_traces(DC_pulse_sweep_parameter.data['acquisition_traces']['output'])

TypeError: 'NoneType' object is not subscriptable

## Tracking a peak

In [4]:
y_sweep = create_set_vals(step_percentage=30, min_val=TGAC(), max_val=0.5, set_parameter=TGAC)

In [82]:
y_sweep = TG[2.05:2.06:0.001]

In [83]:
from silq.parameters.measurement_parameters import TrackPeakParameter
track_peak_parameter = TrackPeakParameter('track_peak', set_parameter=DF_DS, 
                                          acquisition_parameter=DC_parameter, 
                                          step_percentage=20, peak_width=0.015)

In [84]:
DC_parameter.setup()
data = qc.Loop(
    y_sweep).each(
        track_peak_parameter).then(
                qc.Task(layout.stop)).run(
    name='track_peak', progress_interval=True)

DataSet:
data = qc.load_data('2017-01-05/#003_track_peak_12-20-12')
   mode = DataMode.PULL_FROM_SERVER
   <Type>   | <array_id>       | <array.name>     | <array.shape>
   Setpoint | TG_set           | TG               | (10,)
   Measured | index0           | index0           | (10, 14)
   Measured | optimal_set_vals | optimal_set_vals | (10,)
   Measured | DF_DS_set        | DF_DS_set        | (10, 14)
   Measured | DC_voltage       | DC_voltage       | (10, 14)
started at 2017-01-05 12:20:16


In [85]:
data.sync()
plot = qc.MatPlot(subplots=2)
plot[0].add(data.DC_voltage)
plot[1].add(data.optimal_set_vals)

In [ ]:
data.parameter_info[]

In [7]:
DC_parameter.setup()
track_peak_parameter()
layout.stop()
plt.figure()
plt.plot(track_peak_parameter.result[1], track_peak_parameter.result[2])

# EPR measurements (no steering/ESR)

This is a calibration routine to find a point with high contrast (difference in signal between up state and down state). 
This measurement consists of three stages: empty, load, and read.
In the empty stage, the donor atom is ionized. 
It is then loaded with an electron in a random state.
Finally the state is read out for a long duration.
The contrast is found by looking if blips occured at the beginning of the read trace and at the end of the read trace.
The contrast is the difference in percentage of blips occuring at the beginning versus at the end.
The end part of the read trace is related to the number of dark counts, since we assume that an up-state electron will have had enough time to have tunneled off and on again.

## EPR - 2D scan

In [ ]:
DF_DS(0.546)
TGAC(0.219)

In [93]:
y_sweep, x_sweep = create_set_vals(2, step_percentage=30, points=8)
#y_sweep, x_sweep = create_set_vals('2D', spans=0, nums=5)

TypeError: create_set_vals() got an unexpected keyword argument 'spans'

In [ ]:
EPR_parameter.settings(samples=150)
data = qc.Loop(
    y_sweep).loop(
    x_sweep).each(
    EPR_parameter).run(
    name='EPR_calibration', progress_interval=True)

In [ ]:
plot = qc.MatPlot(subplots=(1, 3), figsize=(12, 4))
plot.add(data.contrast, subplot=1)
plot.add(data.dark_counts, subplot=2)
plot.add(data.voltage_difference, subplot=3)
plt.tight_layout()

## EPR - Single point

In [ ]:
EPR_parameter.single_settings(samples=150, silent=False)
EPR_parameter();

# Adiabatic sweep measurements

## ESR Adiabatic - Center frequency scan

This code sweeps over the adiabatic sweep center frequency to find the ESR frequency.

To find the ESR frequency, use 28GHz * B0 as starting point, and search around that frequency.
Use a big initial frequency_deviation (around 60 MHz) and long duration (0.5 ms)
Once you see a spike in contrast, center around there and decrease both deviation and duration.

In [4]:
adiabatic_ESR_parameter.pulse_sequence['adiabatic'].duration=0.15
adiabatic_ESR_parameter.pulse_sequence['adiabatic'].frequency_deviation = 1e6
adiabatic_ESR_parameter.pulse_sequence['adiabatic']

FrequencyRampPulse(adiabatic (ESR), f_center=27951.00 MHz, f_dev=1.00, power=0, t_start=3, t_stop=3.15)

In [58]:
f_center = properties_config['frequency_ESR']
f_span = 2e6
f_step = 0.1e6
adiabatic_sweep_vals = list(np.arange(f_center-f_span/2, f_center+f_span/2, f_step))
print('Sweep vals: {}'.format(len(adiabatic_sweep_vals)))

Sweep vals: 20


In [63]:
adiabatic_ESR_parameter.samples=40
data = qc.Loop(
    adiabatic_ESR_parameter[adiabatic_sweep_vals]).each(
    qc.Task(partial(print_attr, adiabatic_ESR_parameter, 'pulse_sequence')),
    adiabatic_ESR_parameter).run(
    name='adiabatic_ESR_f_center', progress_interval=True)
plot = qc.MatPlot(data.contrast, use_offset=False)

DataSet:
   mode     = DataMode.PULL_FROM_SERVER
   location = '2016-12-24/#037_adiabatic_ESR_f_center_18-34-58'
   <Type>   | <array_id>         | <array.name>       | <array.shape>
   Setpoint | adiabatic_ESR_set  | adiabatic_ESR      | (20,)
   Measured | contrast           | contrast           | (20,)
   Measured | dark_counts        | dark_counts        | (20,)
   Measured | voltage_difference | voltage_difference | (20,)
started at 2016-12-24 18:35:00


## NMR adiabatic frequency scan

In [ ]:
#NMR sweep frequencies
f_center = 66.82e6
f_span = 3e6
f_step = 20e3
NMR_vals = list(np.arange(f_center-f_span/2, f_center+f_span/2, f_step))
print('Sweep vals: {}'.format(len(NMR_vals)))

In [ ]:
data = qc.Loop(adiabatic_NMR_parameter[NMR_vals]
              ).each(qc.Task(adiabatic_NMR_parameter.setup),
                     adiabatic_NMR_parameter,
                     qc.Task(partial(adiabatic_ESR_parameter, frequency_center_ESR['up'])),
                     adiabatic_ESR_parameter,
                     qc.Task(partial(adiabatic_ESR_parameter, frequency_center_ESR['down'])),
                     adiabatic_ESR_parameter,
                    ).run(name='NMR_adiabatic_sweep', progress_interval=True)

In [ ]:
plot = qc.MatPlot(data.contrast_3_4)
plot.add(data.contrast_5_4)

## Adiabatic - frequency deviation versus ESR duration

This is a 2D scan of frequency deviation versus duration.
From this measurement, the minimum acceptable values for frequency deviation and ESR duration can be extracted.

In [44]:
pulse = adiabatic_ESR_parameter.pulse_sequence['adiabatic']
adiabatic_deviation_parameter = general_parameters.AttributeParameter(object=pulse, attribute='frequency_deviation')
adiabatic_duration_parameter = general_parameters.AttributeParameter(object=pulse, attribute='duration')

adiabatic_deviation_vals = list(np.arange(0.5e6, 2.1e6, 0.25e6))
adiabatic_duration_vals = list(np.arange(0.075, 0.251, 0.025))

In [50]:
adiabatic_ESR_parameter.samples = 400
data = qc.Loop(
    adiabatic_deviation_parameter[adiabatic_deviation_vals]).loop(
    adiabatic_duration_parameter[adiabatic_duration_vals]).each(
    qc.Task(partial(print_attr, adiabatic_ESR_parameter, 'pulse_sequence')),
    adiabatic_ESR_parameter).run(
    name='adiabatic_deviation_vs_duration', progress_interval=True)

DataSet:
   mode     = DataMode.PULL_FROM_SERVER
   location = '2016-12-24/#034_adiabatic_deviation_vs_duration_17-17-26'
   <Type>   | <array_id>              | <array.name>        | <array.shape>
   Setpoint | frequency_deviation_set | frequency_deviation | (7,)
   Setpoint | duration_set            | duration            | (7, 8)
   Measured | contrast                | contrast            | (7, 8)
   Measured | dark_counts             | dark_counts         | (7, 8)
   Measured | voltage_difference      | voltage_difference  | (7, 8)
started at 2016-12-24 17:17:28


In [51]:
plot = qc.MatPlot(subplots=(1,2), figsize=(12,5), interval=10)
plot.add(data.contrast, subplot=1, nticks=6)
plot.add(data.dark_counts, subplot=2, nticks=6)
# for ax in plot.subplots:
#     ax.set_xscale('log')
#     ax.set_yscale('log')
plot.fig.tight_layout(rect=[0, 0.03, 1, 0.95])

## Adiabatic - 2D scan

In [30]:
# TGAC, DF_DS sweep
y_sweep, x_sweep = create_set_vals(2, step_percentage=12, points=10)

In [32]:
DF_DS(0.47262)
TGAC(0.39276)

In [33]:
adiabatic_ESR_parameter()

[0.46696969696969703, 0.23000000000000001, 0.7112762706261333]

## Adiabatic - 2D deviation versus duration

In [ ]:
pulse = adiabatic_ESR_parameter.pulse_sequence['adiabatic_sweep']
adiabatic_deviation_parameter = general_parameters.AttributeParameter(object=pulse, attribute='frequency_deviation')
adiabatic_duration_parameter = general_parameters.AttributeParameter(object=pulse, attribute='duration')


adiabatic_ESR_parameter.setup(samples=250, readout_threshold_voltage=readout_threshold_voltage,
                                save_traces=False)
data = qc.Loop(dummy_parameter[0:100:1]
              ).each(qc.Task(adiabatic_deviation_parameter, 5e6),
                     qc.Task(adiabatic_duration_parameter, .15),
                     qc.Task(adiabatic_ESR_parameter.setup),
                     adiabatic_ESR_parameter,
                     qc.Task(adiabatic_deviation_parameter, 15e6),
                     qc.Task(adiabatic_duration_parameter, .35),
                     qc.Task(adiabatic_ESR_parameter.setup),
                     adiabatic_ESR_parameter,
                    ).run(name='adiabatic_calibration',
                          progress_interval=True)

In [ ]:
plot = qc.MatPlot(data.contrast_3_4);
plot.add(data.contrast_7_4)

## Flip nucleus

In [ ]:
# Try to flip the nucleus
adiabatic_NMR_parameter(frequency_center_NMR)
adiabatic_NMR_parameter.samples = 1
adiabatic_NMR_parameter();

## Find ESR frequency

In [ ]:
select_ESR_parameter.setup(samples=50, print_results=True, readout_threshold_voltage=readout_threshold_voltage)
data = qc.Loop(dummy_parameter[0:100:1]).each(qc.Task(adiabatic_NMR_parameter.setup),
                                             adiabatic_NMR_parameter,
                                             qc.Task(select_ESR_parameter.setup),
                                             select_ESR_parameter,
                                             qc.Task(adiabatic_ESR_parameter, partial(getattr, select_ESR_parameter, 'frequency_ESR')),
                                             qc.Task(adiabatic_deviation_parameter, 5e6),
                                             qc.Task(adiabatic_duration_parameter, .15),
                                             qc.Task(adiabatic_ESR_parameter.setup),
                                             adiabatic_ESR_parameter,
                                             qc.Task(adiabatic_deviation_parameter, 15e6),
                                             qc.Task(adiabatic_duration_parameter, .35),
                                             qc.Task(adiabatic_ESR_parameter.setup),
                                             adiabatic_ESR_parameter,
                                            ).run(name='test_find_ESR',
                                                 progress_interval=True)

In [ ]:
plot = qc.MatPlot(data.contrast_up, label='nucleus up')
plot.add(data.contrast_down, label='nucleus down')
plot.add(data.contrast_8_4, label='ESR narrow')
plot.add(data.contrast_12_4, label='ESR wide')

# Rabis

## Find Rabi frequency

In [27]:
#determine Rabi frequency
rabi_ESR_parameter.pulse_sequence['rabi'].duration=0.1
rabi_ESR_parameter.pulse_sequence['rabi'].power=0
rabi_ESR_parameter.pulse_sequence['rabi'].t_start=3

f_center = 27.950885000e9 #properties_config['frequency_ESR']
rabi_ESR_parameter.pulse_sequence['rabi'].frequency = f_center
f_span = 5e6
f_step = 0.01e6
rabi_sweep_vals = list(np.arange(f_center-f_span/2, f_center+f_span/2, f_step))
print('Sweep vals: {}'.format(len(rabi_sweep_vals)))
rabi_ESR_parameter.pulse_sequence

Sweep vals: 500


PulseSequence with 5 pulses, duration: 27
	DCPulse(plunge, A=2, t_start=0, t_stop=5)
	SinePulse(rabi (ESR), f=27950.88 MHz, power=0, t_start=3, t_stop=3.1)
	DCPulse(read, A=0, t_start=5, t_stop=25)
	DCPulse(final, A=0, t_start=25, t_stop=27)
	
	Disabled pulses:
	SteeredInitialization(steered_initialization, t_no_blip=120 ms, t_max_wait=400, t_buffer=30, V_th=0.3333333333333333)

In [28]:
rabi_ESR_parameter.samples=50
data = qc.Loop(
    rabi_ESR_parameter[rabi_sweep_vals]).each(
    rabi_ESR_parameter).run(
    name='rabi_ESR_f_center', progress_interval=True)
plot = qc.MatPlot(data.contrast, use_offset=False)

DataSet:
data = qc.load_data('2017-01-13/#019_rabi_ESR_f_center_22-58-19')
   mode = DataMode.PULL_FROM_SERVER
   <Type>   | <array_id>         | <array.name>       | <array.shape>
   Setpoint | rabi_ESR_set       | rabi_ESR           | (500,)
   Measured | contrast           | contrast           | (500,)
   Measured | dark_counts        | dark_counts        | (500,)
   Measured | voltage_difference | voltage_difference | (500,)
started at 2017-01-13 22:58:23


In [28]:
max_val = data.contrast.argmax()
rabi_sweep_vals[max_val]

27950885000.0

In [16]:
rabi_frequency = 27950885000.0

## Rabi drive

In [19]:
rabi_drive_ESR_parameter.pulse_sequence['rabi'].duration=0.1
rabi_drive_ESR_parameter.pulse_sequence['rabi'].power=0
rabi_drive_ESR_parameter.pulse_sequence['rabi'].t_start=3

f_center = rabi_frequency #properties_config['frequency_ESR']
rabi_drive_ESR_parameter.pulse_sequence['rabi'].frequency = f_center
drive_duration_start = 0
drive_duration_stop = 0.1
drive_duration_step = 0.0005
drive_duration_sweep_vals = list(np.arange(drive_duration_start, drive_duration_stop, drive_duration_step))
print('Sweep vals: {}'.format(len(drive_duration_sweep_vals)))
rabi_drive_ESR_parameter.pulse_sequence

Sweep vals: 200


PulseSequence with 5 pulses, duration: 27
	DCPulse(plunge, A=2, t_start=0, t_stop=5)
	SinePulse(rabi (ESR), f=27950.88 MHz, power=0, t_start=3, t_stop=3.1)
	DCPulse(read, A=0, t_start=5, t_stop=25)
	DCPulse(final, A=0, t_start=25, t_stop=27)
	
	Disabled pulses:
	SteeredInitialization(steered_initialization, t_no_blip=120 ms, t_max_wait=400, t_buffer=30, V_th=0.3333333333333333)

In [31]:
rabi_drive_ESR_parameter.samples=80
data = qc.Loop(
    rabi_drive_ESR_parameter[drive_duration_sweep_vals]).each(
    rabi_drive_ESR_parameter).run(
    name='rabi_drive_ESR', progress_interval=True)
plot = qc.MatPlot(data.contrast, use_offset=False)

DataSet:
data = qc.load_data('2017-01-13/#005_rabi_drive_ESR_13-07-36')
   mode = DataMode.PULL_FROM_SERVER
   <Type>   | <array_id>         | <array.name>       | <array.shape>
   Setpoint | rabi_drive_ESR_set | rabi_drive_ESR     | (100,)
   Measured | contrast           | contrast           | (100,)
   Measured | dark_counts        | dark_counts        | (100,)
   Measured | voltage_difference | voltage_difference | (100,)
started at 2017-01-13 13:07:39


In [36]:
max_val = data.contrast.argmax()
pulse_length = drive_duration_sweep_vals[max_val]
pi_pulse_length =0.007
pulse_length/2


0.010999999999999999

# Dark counts

In [ ]:
dark_counts_parameter.single_settings(samples=150, silent=True)
dark_counts_parameter();

## Dark counts vs plunge voltage

In [13]:
dark_counts_plunge_parameter.samples=50
adiabatic_ESR_parameter.samples=50

T1_parameter(10)
data = qc.Loop(
    dummy_parameter[0:20:1]).loop(
        plunge_voltage_parameter[1:2.5:0.1]).each(
            dark_counts_plunge_parameter,
            adiabatic_ESR_parameter).run(
    name='T1_plunge_voltage', progress_interval=True)

DataSet:
   mode     = DataMode.PULL_FROM_SERVER
   location = '2016-12-29/#026_T1_plunge_voltage_15-36-23'
   <Type>   | <array_id>           | <array.name>       | <array.shape>
   Setpoint | dummy_set            | dummy              | (20,)
   Setpoint | plunge_amplitude_set | plunge_amplitude   | (20, 15)
   Measured | up_proportion        | up_proportion      | (20, 15)
   Measured | num_traces           | num_traces         | (20, 15)
   Measured | contrast             | contrast           | (20, 15)
   Measured | dark_counts          | dark_counts        | (20, 15)
   Measured | voltage_difference   | voltage_difference | (20, 15)
started at 2016-12-29 15:36:25


In [22]:
plot = qc.MatPlot(subplots=(1,3), figsize=(12,4))
plot.add(data.up_proportion, subplot=0)
plot.add(data.contrast, subplot=1)
plot.add(np.nanmean(data.contrast, 0), x=data.plunge_amplitude_set[0], subplot=2, label='contrast')
plot.add(np.nanmean(data.up_proportion, 0), x=data.plunge_amplitude_set[0], subplot=2, label='dark counts')
plot[2].set_xlabel('plunge_amplitude')
plot[2].legend()
plot.tight_layout()

# Read level sweep (to be fixed)

In [ ]:
variable_read_parameter.setup(samples=100)
data = qc.Loop(variable_read_parameter[-1.5:2.5:0.04]
                     ).each(variable_read_parameter
                           ).run(name='variable_read',
                                progress_interval=True)

In [ ]:
plotQ = qc.QtPlot()
plotQ.add(data.Channel_A_signal)

plotQ = qc.QtPlot()
plotQ.add(data.Channel_C_signal)

# T1 measurement

## T1 - Single point

In [ ]:
T1_parameter(20000)
T1_parameter.single_settings(samples=50)
%time up_proportion, num_traces = T1_parameter()

In [ ]:
up_proportion

In [ ]:
plot_traces(T1_parameter.trace_segments['output']['read'])

## T1 - single scan

In [5]:
# Calculate T1 durations
T1_wait_times = list(np.logspace(1, 3.5, num=10, base=10))

# Shuffle times
#np.random.shuffle(T1_wait_times)
print('Shuffled T1 wait times: {}'.format(T1_wait_times))

Shuffled T1 wait times: [10.0, 18.957356524063755, 35.938136638046274, 68.129206905796153, 129.15496650148839, 244.84367468222268, 464.15888336127819, 879.92254356910746, 1668.1005372000591, 3162.2776601683795]


In [6]:
# Single T1 sweep
T1_parameter.samples=50
data = qc.Loop(
    T1_parameter[T1_wait_times]).each(
    T1_parameter).run(name='T1_single_sweep')

plot = qc.MatPlot()
plot.add(data.up_proportion)
plot[0].set_xscale('log')

DataSet:
data = qc.load_data('2017-01-09/#010_T1_single_sweep_17-07-22')
   mode = DataMode.PULL_FROM_SERVER
   <Type>   | <array_id>           | <array.name>     | <array.shape>
   Setpoint | T1_wait_time_ESR_set | T1_wait_time_ESR | (10,)
   Measured | up_proportion        | up_proportion    | (10,)
   Measured | num_traces           | num_traces       | (10,)
started at 2017-01-09 17:07:26


C:\Users\lab.BluG12Meas\Anaconda3\lib\site-packages\matplotlib\axes\_base.py:2899: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=10.0, right=10.0
  'left=%s, right=%s') % (left, right))


## T1 - Multiple scans

In [90]:
# Calculate T1 durations (shuffled)
T1_wait_times = list(np.logspace(1, 4.2, num=12, base=10))
np.random.shuffle(T1_wait_times)
print('Shuffled T1 wait times: {}'.format(T1_wait_times))

Shuffled T1 wait times: [19.539304046896113, 4151.2780027522995, 74.598015234879369, 38.178440263705077, 15848.931924611141, 10.0, 1087.3356727196567, 145.75933009692972, 8111.3083078968812, 284.80358684358049, 556.48638769832951, 2124.5782310305694]


In [93]:
T1_parameter.samples = 50
dark_counts_parameter.samples = 100
adiabatic_ESR_parameter.samples = 100
data = qc.Loop(
    dummy_parameter[0:80:1]).each(
    adiabatic_ESR_parameter,
    dark_counts_parameter,
    qc.Loop(T1_parameter[T1_wait_times]).each(T1_parameter),
                    ).run(name='T1_data', progress_interval=True)

DataSet:
   mode = DataMode.PULL_FROM_SERVER
   data = qc.load_data('2016-12-30/#053_T1_data_23-04-20')
   <Type>   | <array_id>           | <array.name>       | <array.shape>
   Setpoint | dummy_set            | dummy              | (80,)
   Measured | contrast             | contrast           | (80,)
   Measured | dark_counts_0_1      | dark_counts        | (80,)
   Measured | voltage_difference   | voltage_difference | (80,)
   Measured | dark_counts_1        | dark_counts        | (80,)
   Setpoint | T1_wait_time_ESR_set | T1_wait_time_ESR   | (80, 12)
   Measured | up_proportion        | up_proportion      | (80, 12)
   Measured | num_traces           | num_traces         | (80, 12)
started at 2016-12-30 23:04:25


In [99]:
plot = qc.MatPlot(subplots=(1,3), figsize=(12,3.5))
plot.add(data.up_proportion, subplot=0)
# plot.subplots[0].set_xscale('log')
# plot.add(data.T1_wait_time_set[0], np.nanmean(data.up_proportion_5_0_0, axis=0), subplot=2)
# plot.subplots[1].set_xscale('log')
# plot.subplots[1].set_xlim(10, 13000)
plot.add(data.contrast, subplot=1)
plot.add(data.dark_counts_1, subplot=2)
# plot.add(data.dark_counts_0_1, subplot=3)
plt.tight_layout()

## T1 - Plunge voltage scan
This scan measures the contrast for a fixed wait time as the plunge voltage is varied

### Fixed-tau plunge voltage scan

In [90]:
T1_parameter.samples=50

T1_parameter(2500)
data = Loop(
    dummy_parameter[0:10:1]).each(
        adiabatic_ESR_parameter,
        Loop(plunge_voltage_parameter[1:2.6:0.25]).each(
            T1_parameter)).run(
    name='T1_plunge_voltage', progress_interval=True)

DataSet:
   mode = DataMode.PULL_FROM_SERVER
   data = qc.load_data('2016-12-30/#026_T1_plunge_voltage_17-01-15')
   <Type>   | <array_id>           | <array.name>       | <array.shape>
   Setpoint | dummy_set            | dummy              | (10,)
   Measured | contrast             | contrast           | (10,)
   Measured | dark_counts          | dark_counts        | (10,)
   Measured | voltage_difference   | voltage_difference | (10,)
   Setpoint | plunge_amplitude_set | plunge_amplitude   | (10, 7)
   Measured | up_proportion        | up_proportion      | (10, 7)
   Measured | num_traces           | num_traces         | (10, 7)
started at 2016-12-30 17:01:19


In [91]:
plot = qc.MatPlot(subplots=3)
plot.add(data.up_proportion, subplot=0)
plot.add(np.nanmean(data.up_proportion, 0), x=data.plunge_amplitude_set[0], subplot=1)
plot[1].set_xlabel('amplitude')
plot.add(data.contrast, subplot=2, label='contrast')
plot.add(data.dark_counts, subplot=2, label='dark counts')
plot[2].legend()

C:\Users\lab.BluG12Meas\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


### Detailed plunge voltage scan

In [10]:
# Calculate T1 durations
T1_wait_times = list(np.logspace(1, 4, num=12, base=10))
print('T1 wait times: {}'.format(T1_wait_times))

T1 wait times: [10.0, 18.738174228603839, 35.111917342151308, 65.793322465756788, 123.28467394420659, 231.01297000831579, 432.87612810830569, 811.13083078968725, 1519.9110829529332, 2848.0358684357989, 5336.6992312063076, 10000.0]


In [11]:
T1_parameter.samples = 30
dark_counts_plunge_parameter.samples = 100
adiabatic_ESR_parameter.samples = 100

data = Loop(
    dummy_parameter[0:30:1]).loop(
        plunge_voltage_parameter[1:3.6:0.25]).each(
            dark_counts_plunge_parameter,
            adiabatic_ESR_parameter,
            Loop(T1_parameter[T1_wait_times]).each(
                T1_parameter)).run(
    name='T1_plunge_voltage', progress_interval=True)

DataSet:
data = qc.load_data('2017-01-09/#011_T1_plunge_voltage_17-16-53')
   mode = DataMode.PULL_FROM_SERVER
   <Type>   | <array_id>           | <array.name>       | <array.shape>
   Setpoint | dummy_set            | dummy              | (30,)
   Setpoint | plunge_amplitude_set | plunge_amplitude   | (30, 11)
   Measured | up_proportion_0_0    | up_proportion      | (30, 11)
   Measured | num_traces_0_1       | num_traces         | (30, 11)
   Measured | contrast             | contrast           | (30, 11)
   Measured | dark_counts          | dark_counts        | (30, 11)
   Measured | voltage_difference   | voltage_difference | (30, 11)
   Setpoint | T1_wait_time_ESR_set | T1_wait_time_ESR   | (30, 11, 12)
   Measured | up_proportion_2_0_0  | up_proportion      | (30, 11, 12)
   Measured | num_traces_2_0_1     | num_traces         | (30, 11, 12)
started at 2017-01-09 17:16:56


In [12]:
plot = qc.MatPlot(subplots=(1,3), figsize=(12,4))
# plot.add(data.up_proportion, subplot=1)
plot.add(np.nanmean(data.up_proportion_2_0_0, 0),subplot=0)
plot.add(data.contrast, subplot=1)
plot.add(data.dark_counts, subplot=2)

C:\Users\lab.BluG12Meas\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


In [24]:
config['user'].pulses

{'adiabatic_ESR': {'duration': 0.2,
  'frequency_deviation': 500000.0,
  'power': 0,
  't_start': 3},
 'adiabatic_NMR': {'duration': 0.5,
  'frequency_deviation': 100000.0,
  'power': 0,
  't_start': 3},
 'empty': {'amplitude': -1.5, 'duration': 5},
 'final': {'amplitude': 0, 'duration': 2},
 'plunge': {'amplitude': 2, 'duration': 5},
 'rabi': {'duration': 0.1, 'power': 0, 't_start': 3},
 'read': {'amplitude': 0, 'duration': 20},
 'read_long': {'duration': 120},
 'steered_initialization': {'enabled': False,
  't_buffer': 30,
  't_max_wait': 400,
  't_no_blip': 120},
 'steered_initialization_long': {'t_max_wait': 1000, 't_no_blip': 200}}

# Notes

## Acquisition parameters

Acquisition parameters are used to perform simple measurements containing a single pulse sequence.
Examples are DCParameter, AdiabaticSweepParameter, DarkCountsParameter, etc.

Acquisition parameters are run by performing by calling the parameter (parameter(), which performs a parameter.get() command).
At the start of a get(), the pulse sequence is sent to the Layout, which sets up all the instruments.
The exception to this is the DC parameter, which needs a separate setup at the start of a measurement.

## Config settings

In many cases, settings for parameters and pulses are setup-specific.
To keep the SilQ software general, these settings have been moved to the QCoDeS config.
The config is imported as 

`from qcodes import config`

The config is saved to a file, and has several save locations. We usually used the current working directory (cwd) for saving/loading.
For more info see config in qcodes/docs/examples.

Special care must be taken when combining the config with multiprocessing.
When a new process is started, it will load the config from the physical file.
When changing settings in the config, it is therefore necessary to perform a config.save_to_cwd() if you want the changes to be reflected in any multiprocessing measurements.

Several objects (including AcquisitionParameters and pulses) are subclasses of SettingsClass.
The SettingsClass overrides `__getattribute__`, which means that when requesting an attribute (obj.attr), the following steps are undertaken.
1. If attr is a key in either `obj._single_settings`, or `obj._temporary_settings`, return that value (see below for more information).
2. If obj.attr does not exist, raise an AttributeError as usual.
3. If the attribute exists and is not equal to None, return that value.
4. If the attribute exists but is equal to None, search for its value in the respective config*:
    1. If obj.mode is not None and 1{attr}_{mode}1 is a key in the config, return value
    2. If attr is key in the config, return that value
    
By default, the respective config is `config['user']['properties']`. 
However, for pulses, the respective config is `config['user']['pulses'][{pulse_name}]`

Attributes can be temporarily overriden through `obj.single_settings(attr1=val1, ...)` and `obj.temporary_settings(attr1=val1, ...)`.
Single settings are always cleared after an obj.get() command, and so are useful when you for instance want to do a single measurement with different settings than usual. Temporary settings are in effect until you perform an obj.clear_settings() command.

To enable/disable steered initialization, use:

adiabatic_sweep_parameter.steered_intialization = True/False

This adds/removes the steered initialization "pulse" from the EPR_parameter.pulse_sequence
If steered_initialization is enabled, you can access and modify its properties via

steered_initialization = adiabatic_sweep_parameter.pulse_sequence['steered_initialization']

Note that steered initialization is currently only implemented for the adiabatic sweep, but is easy to add for other parameters, such as T1_parameter (just add the pulse)

steered_controller is the acquisition controller specifically for steered initialization
You can look at the initialization traces when using steered initialization.

First set the flag 
steered_controller.record_initialization_traces(True)

The initialization traces can then be accessed
steered_controller.initialization_traces()

You can also look at the first buffer after the pulseblaster.start() command has been set via
steered_controller.post_initialization_traces()
This can be useful for checking the delay between sending the command and plunging

To setup a parameter where steered initialization is enabled, you must pass it a readout threshold voltage every time.

adiabatic_sweep_parameter.setup(readout_threshold_voltage=threshold_voltage)

This threshold voltage can be obtained from an EPR scan (see T1_measurement):

EPR_parameter.setup(samples=30)
EPR_parameter()
traces_read = EPR_parameter.trace_segments['output']['read']
_, _, readout_threshold_voltage = analysis.find_high_low(traces_read)

In [ ]:
adiabatic_sweep_parameter.pulse_sequence.add(adiabatic_sweep_parameter._steered_initialization)
adiabatic_sweep_parameter.pulse_sequence.remove(adiabatic_sweep_parameter._steered_initialization)

To save raw traces, use:

EPR_parameter.setup(save_traces=True, data_manager=data_manager_raw)

In [ ]:
plot = CalibrationPlot(data)

# Measurements run from Slack

In [31]:
adiabatic_ESR_parameter.samples = 100
data = qc.Loop(
        y_sweep).loop(
            x_sweep).each(
                adiabatic_ESR_parameter).run(
        name='adiabatic_calibration', progress_interval=True)
CalibrationPlot(data)

DataSet:
data = qc.load_data('2017-01-16/#001_adiabatic_calibration_10-40-58')
   mode = DataMode.PULL_FROM_SERVER
   <Type>   | <array_id>         | <array.name>       | <array.shape>
   Setpoint | TGAC_set           | TGAC               | (10,)
   Setpoint | DF_DS_set          | DF_DS              | (10, 10)
   Measured | contrast           | contrast           | (10, 10)
   Measured | dark_counts        | dark_counts        | (10, 10)
   Measured | voltage_difference | voltage_difference | (10, 10)
started at 2017-01-16 10:41:07


In [176]:
DCPlot(data);

In [182]:
# TGAC, DF_DS sweep
y_sweep, x_sweep = create_set_vals(2, step_percentage=12, points=8)

In [183]:
adiabatic_ESR_parameter.samples = 100
data = qc.Loop(
    y_sweep).loop(
        x_sweep).each(
            adiabatic_ESR_parameter).run(
    name='adiabatic_calibration', progress_interval=True)

CalibrationPlot(data)

DataSet:
data = qc.load_data('2017-01-09/#006_adiabatic_calibration_15-27-12')
   mode = DataMode.PULL_FROM_SERVER
   <Type>   | <array_id>         | <array.name>       | <array.shape>
   Setpoint | TGAC_set           | TGAC               | (8,)
   Setpoint | DF_DS_set          | DF_DS              | (8, 8)
   Measured | contrast           | contrast           | (8, 8)
   Measured | dark_counts        | dark_counts        | (8, 8)
   Measured | voltage_difference | voltage_difference | (8, 8)
started at 2017-01-09 15:27:16


In [19]:
adiabatic_ESR_parameter.samples = 100
data = qc.Loop(y_sweep).loop(x_sweep).each(
    adiabatic_ESR_parameter).run(
    name='adiabatic_calibration', progress_interval=True)
CalibrationPlot(data)

DataSet:
data = qc.load_data('2017-01-13/#014_adiabatic_calibration_15-11-06')
   mode = DataMode.PULL_FROM_SERVER
   <Type>   | <array_id>         | <array.name>       | <array.shape>
   Setpoint | TGAC_set           | TGAC               | (12,)
   Setpoint | DF_DS_set          | DF_DS              | (12, 12)
   Measured | contrast           | contrast           | (12, 12)
   Measured | dark_counts        | dark_counts        | (12, 12)
   Measured | voltage_difference | voltage_difference | (12, 12)
started at 2017-01-13 15:11:10


In [13]:
DCPlot(data)

In [35]:
DCPlot(data)